# Inleveropgave 2: Model-Free Prediction and Control

### Sources
- https://towardsdatascience.com/reinforcement-learning-rl-101-with-python-e1aa0d37d43b

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List

In [3]:
class Maze:
    def __init__(self, R: np.ndarray, end: List[Tuple[int, int]],
                 p_action: float = 0.7, discount: float = 0.9, threshold: float = 0.0001):
        """The init function.
        
        args:
            R (np.ndarray): The array with the rewards per state.
            end (List[Tuple[int, int]]): A list with the positions of the end states.
        """
        self.R = R
        self.end_states = end

## Model-Free Prediction
### Monte-Carlo Policy Evaluation

In [ ]:
def monte_carlo_policy_evaluation(env: Maze, policy: str = "optimal",
                              discount: float = 0.9, n_episodes: int = 100):
    """"""
    for _ in range(n_episodes):
        pass

    
def generate_episode(env: maze, policy: str):
    """"""
    states, actions, rewards = [], [], []
    
    while True:
        break

### Temporal Difference Learning

In [ ]:
def temporal_difference_learning(env: Maze, policy: str = "optimal",
                                 discount: float = 0.9, n_episodes: int = 100):
    """"""
    for _ in range(n_episodes):
        pass

## Model-Free Control